#### Notebook 2: This Jupyter notebook processes raw authors csv data performing basic data cleaning:

1. Selecting only desired features (raw/derived)
2. String cleaning
3. Handling missing values

In [1]:
# Import Libraries ------------------------------------------------------------
import os
import pandas as pd


# Load Data (only required columns) ---------------------------------------
path = "C:\\Users\\juhic\\OneDrive\\Desktop\\goodreads_kaggle_authors.csv"
authors = pd.read_csv(path, usecols = ['name',
                                     'workcount',
                                     'gender',
                                     'average_rate',
                                     'rating_count',
                                     'review_count',
                                     'country'])


# Rename columns ---------------------------------------------------------
cols = {'name': 'author',
        'workcount': 'work_count',
        'average_rate': 'author_avg_rating',
        'rating_count': 'author_rating_count',
        'review_count': 'author_review_count',
        'country': 'author_country'}
authors.rename(columns = cols, inplace = True)


# String cleaning ------------------------------------------------------
cols = ['author','gender','author_country']

# lowercase, strip spaces from ends
authors[cols] = authors[cols].apply(func = lambda x: x.str.lower().str.strip(), axis = 1)

# strip spaces between ends
for c in cols:
    authors[c] = authors[c].str.replace(r" +", " ")
    
    
# Authors: Drop authors with unknown gender -------------------------------------------------
authors.drop(authors[authors['gender'] == 'unknown'].index, inplace = True)
authors.reset_index(drop = True, inplace = True)

<ipython-input-1-69c1dfecfd0a>:35: FutureWarning: The default value of regex will change from True to False in a future version.
  authors[c] = authors[c].str.replace(r" +", " ")


In [2]:
# Download processed authors data --------------------------------------------------
authors.to_csv('authors_processed.csv')

#### Handling Missing Values (WIP)

In [ ]:
# Fill missing values --------------------------------------------------------------
# Step 1: Identify categorical & numerical columns
cols_categorical = ['gender','author_country']
cols_numerical = ['work_count','author_avg_rating','author_rating_count','author_review_count']

# Step 2: Fillna/Dropna/Impute as needed
authors[cols_categorical] = authors[cols_categorical].fillna('')
authors[cols_numerical] = authors[cols_numerical].fillna(0)